In [2]:
import functools

import numpy as np

import torch
from typing import Dict, List, Optional,  Union, Callable

import pandas as pd

import pyro
import pyro.distributions as dist

import random

from causal_pyro.indexed.ops import IndexSet, gather, indices_of, scatter
from causal_pyro.interventional.handlers import do
from causal_pyro.counterfactual.handlers import MultiWorldCounterfactual, Preemptions

In [3]:
class HalpernPearlModifiedApproximate:

    def __init__(
        self, 
        model: Callable,
        antecedents: Union[Dict[str, torch.Tensor], List[str]],
        outcome: str,
        witness_candidates: List[str],
        observations: Optional[Dict[str, torch.Tensor]],
        sample_size: int = 100,
        event_dim: int = 0
        ):
        
        self.model = model
        self.antecedents = antecedents
        self.outcome = outcome
        self.witness_candidates = witness_candidates
        self.nodes = antecedents + [outcome] + witness_candidates
        self.observations = observations
        self.sample_size = sample_size

        self.antecedents_dict = (
            self.antecedents if isinstance(self.antecedents, dict)
            else self.revert_antecedents(self.antecedents)
        )
    
        self.preemptions = {candidate: functools.partial(self.preempt_with_factual,
                                             antecedents = self.antecedents) for 
                                             candidate in self.witness_candidates}
        

    @staticmethod
    def revert_antecedents(antecedents: List[str]) -> Dict[str, Callable[[torch.Tensor], torch.Tensor]]:
        return {antecedent: (lambda v: 1 - v) for antecedent in antecedents}

    @staticmethod   
    def preempt_with_factual(value: torch.Tensor, *,
                          antecedents: List[str] = None, event_dim: int = 0):
    
        if antecedents is None:
            antecedents = []

        antecedents = [a for a in antecedents if a in indices_of(value, event_dim=event_dim)]

        factual_value = gather(value, IndexSet(**{antecedent: {0} for antecedent in antecedents}),
                                event_dim=event_dim)
            
        return scatter({
            IndexSet(**{antecedent: {0} for antecedent in antecedents}): factual_value,
            IndexSet(**{antecedent: {1} for antecedent in antecedents}): factual_value,
        }, event_dim=event_dim)
        
        
    def __call__(self, *args, **kwargs):
        with pyro.poutine.trace() as trace:
            with MultiWorldCounterfactual():
                with do(actions=self.antecedents_dict):
                    with Preemptions(actions = self.preemptions):
                        with pyro.condition(data={k: torch.as_tensor(v) for k, v in self.observations.items()}):
                            with pyro.plate("plate", self.sample_size):
                                self.consequent = self.model()[self.outcome]
                                self.intervened_consequent = gather(self.consequent, IndexSet(**{ant: {1} for ant in self.antecedents}))
                                self.observed_consequent = gather(self.consequent, IndexSet(**{ant: {0} for ant in self.antecedents}))
                                self.consequent_differs = self.intervened_consequent != self.observed_consequent   
                                pyro.factor("consequent_differs", torch.where(self.consequent_differs, torch.tensor(0.0), torch.tensor(-1e8)))
                            
        self.trace = trace.trace
        self.nodes_trace = {node: self.trace.nodes[node]['value'] for node in self.nodes}
        
        
         # slightly hacky solution for odd witness candidate sets
        if  isinstance(self.consequent_differs.squeeze().tolist(), bool):
            self.existential_but_for = self.consequent_differs.squeeze()
        else:
            #if (len(self.consequent_differs.squeeze().tolist() )>1):
            self.existential_but_for = any(self.consequent_differs.squeeze().tolist()                )  


        witness_dict = dict()
        if self.witness_candidates:
            witness_keys = ["__split_" + candidate for candidate in self.witness_candidates]
            witness_dict = {key: self.trace.nodes[key]['value']  for key in witness_keys}
            

        witness_dict['observed'] = self.observed_consequent.squeeze()
        witness_dict['intervened'] = self.intervened_consequent.squeeze()
        witness_dict['consequent_differs'] = self.consequent_differs.squeeze()

        # slightly hacky as above
        self.witness_df = pd.DataFrame(witness_dict) if self.witness_candidates else witness_dict

        if self.witness_candidates:
            self.witness_df['witness_size'] = self.witness_df[witness_keys].sum(axis = 1)
            satisfactory = self.witness_df[self.witness_df['consequent_differs'] == True]
            
        self.minimal_witness_size = satisfactory['witness_size'].min() if self.witness_candidates else 0
        self.responsibility_internal = 1/(len(self.antecedents) + self.minimal_witness_size)


In [4]:
class HalpernPearlResponsibilityApproximate:

    def __init__(
        self, 
        model: Callable,
        nodes: List,
        antecedent: str,
        outcome: str,
        observations: Dict[str, torch.Tensor], 
        runs_n: int 
    ):
        self.model = model
        self.nodes = nodes
        self.antecedent = antecedent
        self.outcome = outcome
        self.observations = observations
        self.runs_n = runs_n
        
        self.minimal_antecedents_cache = []
        self.antecedent_sizes = []
        self.existential_but_fors = []
        self.minimal_witness_sizes = []
        self.responsibilities = []
        self.HPMs = []

    def __call__(self):
        
        for step in range(1,self.runs_n):

            nodes = self.nodes
            if self.outcome in nodes:
                nodes.remove(self.outcome) 
            
            companion_size = random.randint(0,len(nodes))
            companion_candidates = random.sample(self.nodes, companion_size)
            witness_candidates = [node for node in self.nodes if 
                                node != self.antecedent and 
                                node != self.outcome and 
                                    node not in companion_candidates]

            HPM = HalpernPearlModifiedApproximate(
                model = self.model,
                antecedents = companion_candidates,
                outcome = self.outcome,
                witness_candidates = witness_candidates,
                observations = self.observations,
                sample_size = 1000)
            
            HPM()

            self.HPMs.append(HPM)


            if  HPM.existential_but_for:


                subset_in_cache = any([s.issubset(set(HPM.antecedents)) for s in self.minimal_antecedents_cache])
                if not subset_in_cache:
                    self.minimal_antecedents_cache.append(set(HPM.antecedents))

                    if self.antecedent in HPM.antecedents:
                        self.antecedent_sizes.append(len(HPM.antecedents))
                        self.existential_but_fors.append(HPM.existential_but_for)
                        self.minimal_witness_sizes.append(HPM.minimal_witness_size)
                        self.responsibilities.append(HPM.responsibility_internal)


        self.denumerators = [x + y for x, y in zip(self.antecedent_sizes, self.minimal_witness_sizes)]

        self.responsibilityDF = pd.DataFrame(
            {"existential_but_for": [bool(value) for value in self.existential_but_fors],
                "antecedent_size": self.antecedent_sizes, 
                "minimal_witness_size": self.minimal_witness_sizes,
                "denumerator": self.denumerators,
                "responsibility": self.responsibilities
            }
            )
        if len(self.responsibilityDF['existential_but_for']) == 0:
            self.responsibility = 0
        else:
            min_denumerator = min(self.responsibilityDF['denumerator'])
            self.responsibility = 1/min_denumerator

 

In [5]:
def firing_squad_model():
    probs = pyro.sample("probs", dist.Dirichlet(torch.ones(5)))

    who_has_bullet = pyro.sample("who_has_bullet", dist.OneHotCategorical(probs))

    mark0 = pyro.deterministic("mark0", torch.tensor([who[0] for who in who_has_bullet]), event_dim=0)
    mark1 = pyro.deterministic("mark1", torch.tensor([who[1] for who in who_has_bullet]), event_dim=0)
    mark2 = pyro.deterministic("mark2", torch.tensor([who[2] for who in who_has_bullet]), event_dim=0)
    mark3 = pyro.deterministic("mark3", torch.tensor([who[3] for who in who_has_bullet]), event_dim=0)
    mark4 = pyro.deterministic("mark4", torch.tensor([who[4] for who in who_has_bullet]), event_dim=0)

    dead = pyro.deterministic("dead", mark0 + mark1 + mark2 + mark3 + 
                                mark4  > 0)
    
    return {"probs": probs,
            "mark0": mark0,
            "mark1": mark1,
            "mark2": mark2,

            "mark3": mark3,
            "mark4": mark4, 
            "dead": dead}

In [6]:
# Halpern's strategy: assign weights to models
# and take the weighted average

pyro.set_rng_seed(102)

marksman0_responsibilities = []
for who in range(0,5):
    probs = torch.zeros(5)
    probs[who] = 1.0


    responsibility_marksmen_HPR = HalpernPearlResponsibilityApproximate(
        model = firing_squad_model,
        nodes = ["mark" + str(i) for i in range(0,5)],
        antecedent = "mark0", outcome = "dead",
        observations = {"probs": probs,},
                        runs_n=50)

    responsibility_marksmen_HPR()
    marksman0_responsibilities.append(responsibility_marksmen_HPR.responsibility)

weights = torch.full((5,), .2)

# if your pr(mark0) = i, 
# the blame in this case is i as well 

print(torch.dot(torch.tensor(marksman0_responsibilities), weights))

tensor(0.2000)


## Blame for voters

The general phenomenon in the voting setup: if $a$ voted for and $b$ against, $a>b$, a voter voting for has degree of responsibility $\frac{1}{\lceil (a-b)/2\rceil}$. So much is in Halpern. Now let's go beyond, what if you decide whether to vote and consider your blame if your candidate looses and you didn't vote? Still a somewhat simplistic setup: eight other people are about to vote, each outcome among them you consider equally likely, and you consider the blame associated with you skipping the voting if your candidate fails.

In [109]:
def choose_to_vote_model():
    u_a = pyro.sample("u_a", dist.Categorical(logits=torch.ones(8)))
    u_skip = pyro.sample("u_skip", dist.Bernoulli(0.5))

    a = pyro.deterministic("a", u_a, event_dim=0)
    skip = pyro.deterministic("skip", u_skip, event_dim=0)

    result = pyro.deterministic("result", 
                            (a + 1 - skip) > torch.ceil(9 - skip)/2)

    return {"a": a, "against": 8-a, "voted": 1 - skip, "result": result}

In [123]:
choose_to_vote_model()

{'a': tensor(4),
 'against': tensor(4),
 'voted': tensor(0.),
 'result': tensor(False)}

In [130]:
# your skipping is not an actual cause if a=1

skipping1HPM = HalpernPearlModifiedApproximate(
    model = choose_to_vote_model,
    antecedents = ["skip"],
    outcome = "result",
    witness_candidates = ["a"],
    observations = dict(u_a=1.),
    sample_size = 100)

skipping1HPM()

skipping1HPM.existential_but_for

False

In [131]:
# but it is, if a = 4
skipping4HPM = HalpernPearlModifiedApproximate(
    model = choose_to_vote_model,
    antecedents = ["skip"],
    outcome = "result",
    witness_candidates = ["a"],
    observations = dict(u_a=4.),
    sample_size = 100)

skipping4HPM()

skipping4HPM.existential_but_for

True

In [144]:
# with this model responsibilities follow the same pattern

skipping1HPR = HalpernPearlResponsibilityApproximate(
    model = choose_to_vote_model,
    nodes = ["a", "skip"],
    antecedent = "skip", outcome = "result",
    observations = dict(u_a=1.), 
    runs_n=50
    )

skipping1HPR()
skipping1HPR.responsibility

0

In [143]:
pyro.set_rng_seed(102)
skipping4HPR = HalpernPearlResponsibilityApproximate(
    model = choose_to_vote_model,
    nodes = ["a", "skip"],
    antecedent = "skip", outcome = "result",
    observations = dict(u_a=4.), 
    runs_n=50
    )

skipping4HPR()
print(
skipping4HPR.responsibilityDF,
skipping4HPR.responsibility
)

   existential_but_for  antecedent_size  minimal_witness_size  denumerator  \
0                 True                2                     0            2   
1                 True                1                     0            1   

   responsibility  
0             0.5  
1             1.0   1.0


In [147]:
# but now suppose we work with a wide model

def choosing_to_vote_wide_model():
    u_vote0 = pyro.sample("u_vote0", dist.Bernoulli(0.6))
    u_vote1 = pyro.sample("u_vote1", dist.Bernoulli(0.6))
    u_vote2 = pyro.sample("u_vote2", dist.Bernoulli(0.6))
    u_vote3 = pyro.sample("u_vote3", dist.Bernoulli(0.6))
    u_vote4 = pyro.sample("u_vote4", dist.Bernoulli(0.6))
    u_vote5 = pyro.sample("u_vote5", dist.Bernoulli(0.6))
    u_vote6 = pyro.sample("u_vote6", dist.Bernoulli(0.6))
    u_vote7 = pyro.sample("u_vote7", dist.Bernoulli(0.6))
    u_skip = pyro.sample("u_skip", dist.Bernoulli(0.5))

    vote0 = pyro.deterministic("vote0", u_vote0, event_dim=0)
    vote1 = pyro.deterministic("vote1", u_vote1, event_dim=0)
    vote2 = pyro.deterministic("vote2", u_vote2, event_dim=0)
    vote3 = pyro.deterministic("vote3", u_vote3, event_dim=0)
    vote4 = pyro.deterministic("vote4", u_vote4, event_dim=0)
    vote5 = pyro.deterministic("vote5", u_vote5, event_dim=0)
    vote6 = pyro.deterministic("vote6", u_vote6, event_dim=0)
    vote7 = pyro.deterministic("vote7", u_vote7, event_dim=0)
    skip = pyro.deterministic("skip", u_skip, event_dim=0)


    outcome = pyro.deterministic("outcome", vote0 + vote1 + vote2 + vote3 + 
                                 vote4 + vote5 + vote6 + vote7 + 1 - skip > 4)
    return {"outcome": outcome.float()}

choosing_to_vote_wide_model()

{'outcome': tensor(0.)}

In [149]:
# if a = 1, skipping is still not an actual cause

skipping_wide1HPM = HalpernPearlModifiedApproximate(
    model = choosing_to_vote_wide_model,
    antecedents = ["skip"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(0,8)],
    observations = dict(u_vote0=1., u_vote1=0., u_vote2=0.,
                        u_vote3=0., u_vote4=0., u_vote5=0,
                        u_vote6=0., u_vote7=0.),
    sample_size = 1000)

skipping_wide1HPM()
skipping_wide1HPM.existential_but_for


False

In [150]:
# if a = 4, skipping is  an actual cause

skipping_wide4HPM = HalpernPearlModifiedApproximate(
    model = choosing_to_vote_wide_model,
    antecedents = ["skip"],
    outcome = "outcome",
    witness_candidates = [f"vote{i}" for i in range(0,8)],
    observations = dict(u_vote0=1., u_vote1=1., u_vote2=1.,
                        u_vote3=1., u_vote4=0., u_vote5=0,
                        u_vote6=0., u_vote7=0.),
    sample_size = 1000)

skipping_wide4HPM()
skipping_wide4HPM.existential_but_for


True

In [156]:
# if a = 1 your responsibility is now 1/4
# because if you voted for
# and three other voters changed their mind
# the outcome would be different

skipping_wide1HPR = HalpernPearlResponsibilityApproximate(
    model = choosing_to_vote_wide_model,
    nodes = [f"vote{i}" for i in range(0,8,)] + ["skip"],
    antecedent = "skip", outcome = "outcome",
    observations =  dict(u_vote0=1., u_vote1=0., u_vote2=0.,
                        u_vote3=0., u_vote4=0., u_vote5=0,
                        u_vote6=0., u_vote7=0.), 
    runs_n=500
    )

skipping_wide1HPR()
skipping_wide1HPR.responsibility

0.25

The above illustrates why what Halpern called the naive definition of responsibility used in Halpern is what it is called: it is highly sensitive to modeling choices and needs to be improved. This is a story for another occasion. Meawhile, let's assume that the wide model is the intended one and see what can be said about blame for failure before the voting happens.